In [ ]:
!pip install datasets
!pip install vit-pytorch
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/ColabNotebooks/fit5215-objectdection-s2-2023.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_1957.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_1962.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_1996.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_2038.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_2065.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_2187.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_2240.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_2244.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_2293.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_2295.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_232.JPEG  
  inflating: train_set/train_set/FIT5215_Dataset/ducks/n01855672_2325.JPEG  
  inflating: train_set/tra

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import datasets
import torch
from vit_pytorch import ViT
from PIL import Image

import numpy as np
import pickle
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import datasets
import torch
from vit_pytorch import ViT
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.nn.functional import normalize

from torch import nn
import torchvision
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision

from torchvision.transforms import RandomHorizontalFlip, RandomRotation, RandomCrop, ColorJitter, GaussianBlur
import torch
from torch.optim.optimizer import Optimizer, required
from torch import nn
import torchvision
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision



# For colab
# Load dataset using the datasets library
dataset = datasets.load_dataset("imagefolder", data_dir="/content/train_set/train_set/FIT5215_Dataset")

# Load the test dataset
dataset_test = datasets.load_dataset("imagefolder", data_dir="/content/test_set/test_set/official_testset")
def converter(sample):
    # Convert the image to RGB mode
    sample['image'] = sample['image'].convert('RGB')

    # Resize the image to 224x224 pixels
    sample['image'] = sample['image'].resize((224, 224), Image.ANTIALIAS)

    return sample  # Return the modified sample

dataset = dataset.map(converter)

# Extract data and labels from the dataset
data = dataset["train"]["image"]
targets = dataset["train"]["label"]







Resolving data files:   0%|          | 0/9466 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/16167 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/9466 [00:00<?, ? examples/s]

<ipython-input-7-0ac929bb04a7>:71: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  sample['image'] = sample['image'].resize((224, 224), Image.ANTIALIAS)


In [ ]:
# Define training parameters
image_height = 224
image_width = 224
patch_height = 32
patch_width = 32
batch_size = 64
num_epochs = 7
learning_rate = 0.001
num_classes = 20

def cutmix_or_mixup(inputs, labels, alpha=0.7, cutmix_prob=0.5):
    """
    Applies either CutMix or Mixup augmentation to a batch of input data and labels.

    Args:
        inputs (Tensor): A batch of input data (e.g., images).
        labels (Tensor): Corresponding labels for the input data.
        alpha (float): Mixing parameter for Mixup.
        cutmix_prob (float): Probability of applying CutMix.

    Returns:
        augmented_inputs (Tensor): Augmented input data.
        augmented_labels (Tensor): Augmented labels.
    """
    batch_size = inputs.size(0)
    lam = np.random.beta(alpha, alpha)  # Sample mixing ratio from a beta distribution

    # Generate a random permutation of indices
    index = torch.randperm(batch_size)

    # Decide whether to apply CutMix or Mixup
    if np.random.rand() < cutmix_prob:
        # Apply CutMix
        cut_ratio = np.random.rand()
        cut_h = int(inputs.size(2) * np.sqrt(1 - cut_ratio))
        cut_w = int(inputs.size(3) * np.sqrt(1 - cut_ratio))

        # Create mixed inputs by cutting and pasting two random samples
        mixed_inputs = inputs.clone()
        mixed_inputs[:, :, :cut_h, :cut_w] = inputs[index, :, :cut_h, :cut_w]
        mixed_labels = lam * labels + (1 - lam) * labels[index]
        # Round mixed_labels to the nearest integer
        mixed_labels = torch.round(mixed_labels).long()
    else:
        # Apply Mixup
        # Create mixed inputs by linearly interpolating between two random samples
        mixed_inputs = lam * inputs + (1 - lam) * inputs[index, :]
        mixed_labels = lam * labels + (1 - lam) * labels[index]
        # Round mixed_labels to the nearest integer
        mixed_labels = torch.round(mixed_labels).long()

    return mixed_inputs, mixed_labels

# Access class labels
class_labels = dataset["train"].features["label"].names
# Create DataLoader objects for training and validation data
transform = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

'''transform_1 = transforms.Compose([
    RandomHorizontalFlip(p=0.5),
    RandomRotation(degrees=15),
    ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    GaussianBlur(kernel_size=3)
])'''

# Define your custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None): #,transform_1 = None):
        self.data = data
        self.targets = targets
        self.transform = transform
        #self.transform_1 = transform_1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = self.transform(x)

        '''if self.transform_1:
            x = self.transform_1(x)'''

        return x, y

# Split your data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, targets, test_size=0.3, random_state=42)
train_dataset = CustomDataset(X_train, y_train, transform=transform)
val_dataset = CustomDataset(X_val, y_val, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


model = torchvision.models.vit_b_32(weights=torchvision.models.ViT_B_32_Weights.DEFAULT)
for param in model.parameters():
    param.requires_grad = False
model.heads = nn.Sequential(
    nn.Linear(in_features=model.hidden_dim, out_features=4056),
    nn.ReLU(),
    nn.Linear(in_features=4056, out_features=20)
)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_data, batch_labels in train_loader:
        optimizer.zero_grad()
        # Apply CutMix or Mixup to the batch data and labels
        #mixed_data, mixed_labels = cutmix_or_mixup(batch_data, batch_labels)
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        print(f"Batch Loss {loss.item()}")

    # Calculate and print average training loss for this epoch
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {average_loss:.4f}")


# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch_data, batch_labels in val_loader:
        outputs = model(batch_data)
        _, predicted = torch.max(outputs, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")



dataset_test = dataset_test.map(converter)

# Define your custom dataset class
class CustomDatasetX(Dataset):
    def __init__(self, data,transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]


        if self.transform:
            x = self.transform(x)

        return x

# Extract data and labels from the dataset
data_t = dataset_test["train"]["image"]

test_dataset = CustomDatasetX(data_t,  transform=transform)

test_loader = DataLoader(test_dataset, batch_size=batch_size)

with torch.no_grad():
    with open("my_submission.csv", "w") as f:
        f.write("ID,Label\n")
        for i, data in enumerate(test_loader):
            inputs = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            for j in range(len(predicted)):
                 f.write(f"{i * batch_size + j},{class_labels[predicted[j]]}\n")

print("CSV Printed!")



Batch Loss 3.044137477874756
Batch Loss 2.1444339752197266
Batch Loss 1.4321242570877075
Batch Loss 1.0325907468795776
Batch Loss 0.6168113350868225
Batch Loss 0.4794491231441498
Batch Loss 0.20629502832889557
Batch Loss 0.26037105917930603
Batch Loss 0.17340460419654846
Batch Loss 0.05711732432246208
Batch Loss 0.15316244959831238
Batch Loss 0.16104598343372345
Batch Loss 0.006577222608029842
Batch Loss 0.07059857994318008
Batch Loss 0.2104424089193344
Batch Loss 0.08810615539550781
Batch Loss 0.09189140796661377
Batch Loss 0.038202203810214996
Batch Loss 0.0742562785744667
Batch Loss 0.17189082503318787
Batch Loss 0.10006482899188995
Batch Loss 0.12598824501037598
Batch Loss 0.25435346364974976
Batch Loss 0.10008043795824051
Batch Loss 0.021274346858263016
Batch Loss 0.2999058961868286
Batch Loss 0.07861713320016861
Batch Loss 0.002025354653596878
Batch Loss 0.08810567855834961
Batch Loss 0.021449889987707138
Batch Loss 0.40917539596557617
Batch Loss 0.08924587070941925
Batch Loss 0.

Map:   0%|          | 0/16167 [00:00<?, ? examples/s]

<ipython-input-7-0ac929bb04a7>:71: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  sample['image'] = sample['image'].resize((224, 224), Image.ANTIALIAS)


CSV Printed!
